In [1]:
import pandas as pd
import json
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon

In [2]:
with open('seccensales.JSON') as f:
    features = json.load(f)["features"]

In [3]:
poi_educa = pd.read_csv('educacion_general.csv', sep=',', decimal='.')
poi_lavan = pd.read_csv('lavanderias_general.csv', sep=',', decimal='.')
poi_super = pd.read_csv('supermercado_general.csv', sep=',', decimal='.')
poi_trans = pd.read_csv('transporte_general.csv', sep=',', decimal='.')

In [4]:
poi_educa['duplicator'] = poi_educa['name']+poi_educa['formatted_address']
poi_educa['duplicator2'] = poi_educa['geometry.location.lat']+poi_educa['geometry.location.lng']
poi_educa.drop_duplicates(['duplicator'], inplace=True)
poi_educa.drop_duplicates(['duplicator2'], inplace=True)
poi_educa.shape
poi_educa['type']='education'

In [5]:
poi_lavan['duplicator'] = poi_lavan['name']+poi_lavan['formatted_address']
poi_lavan['duplicator2'] = poi_lavan['geometry.location.lat']+poi_lavan['geometry.location.lng']
poi_lavan.drop_duplicates(['duplicator'], inplace=True)
poi_lavan.drop_duplicates(['duplicator2'], inplace=True)
poi_lavan.shape
poi_lavan['type']='lavanderias'

In [6]:
poi_super['duplicator'] = poi_super['name']+poi_super['formatted_address']
poi_super['duplicator2'] = poi_super['geometry.location.lat']+poi_super['geometry.location.lng']
poi_super.drop_duplicates(['duplicator'], inplace=True)
poi_super.drop_duplicates(['duplicator2'], inplace=True)
poi_super.shape
poi_super['type']='supermercados'

In [7]:
poi_trans['duplicator'] = poi_trans['name']+poi_trans['formatted_address']
poi_trans['duplicator2'] = poi_trans['geometry.location.lat']+poi_trans['geometry.location.lng']
poi_trans.drop_duplicates(['duplicator'], inplace=True)
poi_trans.drop_duplicates(['duplicator2'], inplace=True)
poi_trans.shape
poi_trans['type']='transporte'

In [8]:
poi_df = pd.concat([poi_educa, poi_lavan, poi_super, poi_trans], ignore_index=True)

In [9]:
def check_polygon(lat,long, infeatures):
  seccensal = 0
  point1 = Point(long, lat)
  for feature in infeatures:
      poly = Polygon(feature["geometry"]["coordinates"][0])
      if poly.contains(point1) == True:
          seccensal = feature["properties"]["coddistsec"]
          break
  return seccensal

In [10]:
poi_df["coddistsec"] = poi_df.apply(lambda row: check_polygon(row["geometry.location.lat"], row["geometry.location.lng"], features),axis=1)

In [11]:
poi_df = poi_df[poi_df['coddistsec']!=0.0]

In [12]:
poi_edu = poi_df[poi_df['type']=='education']
poi_edu.shape

(72, 27)

In [13]:
poi_transporte = poi_df[poi_df['type']=='transporte']
poi_transporte.shape

(230, 27)

In [14]:
poi_supermercado = poi_df[poi_df['type']=='supermercados']
poi_supermercado.shape

(77, 27)

In [15]:
poi_lavanderia = poi_df[poi_df['type']=='lavanderias']
poi_lavanderia.shape

(157, 27)

In [16]:
poi_df.shape

(536, 27)

In [17]:

poi_df.drop(['Unnamed: 0','formatted_address','icon','icon_background_color','icon_mask_base_uri','photos','place_id','reference','geometry.viewport.northeast.lat',	'geometry.viewport.northeast.lng',	'geometry.viewport.southwest.lat',	'geometry.viewport.southwest.lng',	'plus_code.compound_code',	'plus_code.global_code',	'opening_hours.open_now',	'permanently_closed',	'duplicator',	'duplicator2'], axis=1, inplace=True)

In [ ]:
poi_df

In [19]:
def divide_string(string_to_analyze):
  new_data = string_to_analyze.split("'")
  return new_data[1]

In [20]:
poi_df["subtypes"] = poi_df.apply(lambda row: divide_string(row["types"]),axis=1)

In [ ]:
poi_df

In [22]:
poi_df.to_csv(r'poi_tratado.csv')